<a href="https://colab.research.google.com/github/RaudelCasas1603/Monky-Detection-/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow_io matplotlib

In [2]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio

# Import the model for training
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [3]:
# Catch the path of the direcotries of the data
DIR_CAPUCHIN = os.path.join('data', 'Parsed_Capuchinbird_Clips')
DIR_NO_CAPUCHIN = os.path.join('data', 'Parsed_Not_Capuchinbird_Clips')

In [4]:
# load_wav_16k_mono: To pass from 44100Hz to 16000hz audio signal
def load_wav_16k_mono(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

# preprocess: To preprocess and gets the spectrogram of each audio file
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [5]:
# Create the dataframes
ds_capuchin = tf.data.Dataset.list_files(DIR_CAPUCHIN + '/*.wav')
ds_no_capuchin = tf.data.Dataset.list_files(DIR_NO_CAPUCHIN + '/*.wav')

In [6]:
# Add the labels
positives = tf.data.Dataset.zip((ds_capuchin, tf.data.Dataset.from_tensor_slices(tf.ones(len(ds_capuchin)))))
negatives = tf.data.Dataset.zip((ds_no_capuchin, tf.data.Dataset.from_tensor_slices(tf.zeros(len(ds_no_capuchin)))))

In [7]:
# Combine the data for the training
ds = positives.concatenate(negatives)

In [8]:
# Map the preprocess function to have the label
ds = ds.map(preprocess)
ds = ds.cache()
ds = ds.shuffle(buffer_size = 1000)  # Shuffle  the data
ds = ds.batch(16)           # Train each 16 samples
ds = ds.prefetch(8)         # Overlaps the preprocessing and model execution of a training step

In [9]:
 # Split into testing and training partions
train = ds.take(36)       # for training
test = ds.skip(36).take(15) # for testing the model

In [10]:
 # Create the AI model 
model = Sequential()
model.add(Conv2D(16, (3, 3), activation = 'relu', input_shape = (1491, 257, 1)))  #  Add the first layer
model.add(Conv2D(16, (3, 3), activation = 'relu'))  # Add the second layer
model.add(Flatten())                                # Add the flatter layer to be one dimension
model.add(Dense(128, activation = 'relu'))          # Add fourth layer with an output of 128 units
model.add(Dense(1, activation = 'sigmoid'))         # Add the last layer with the binary output

In [11]:
# Compile the model using the Adam optimizer
model.compile('Adam', loss = 'BinaryCrossentropy',
              metrics = [tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [12]:
# fetch some data of the model 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1489, 255, 16)     160       
                                                                 
 conv2d_1 (Conv2D)           (None, 1487, 253, 16)     2320      
                                                                 
 flatten (Flatten)           (None, 6019376)           0         
                                                                 
 dense (Dense)               (None, 128)               770480256 
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 770,482,865
Trainable params: 770,482,865
Non-trainable params: 0
_________________________________________________________________


In [3]:
cpus = tf.config.experimental.list_physical_devices('CPU')
print(cpus)
tf.config.set_visible_devices([], 'GPU')  # hide the GPU
tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
 # Train the model
histogram = model.fit(train, epochs = 2, validation_data = test)

Epoch 1/2
